# Projeto de Turismo com LangChain - Passo 3: Criação e Configuração do Router

Este é o notebook final do nosso fluxo de desenvolvimento. Aqui, vamos unir todas as cadeias especializadas que construímos no passo anterior sob um único orquestrador: a `RouterChain`.

O objetivo é criar um sistema que:
1.  Recebe uma pergunta do usuário em linguagem natural.
2.  Usa um LLM para classificar a **intenção** da pergunta.
3.  Direciona a pergunta para a cadeia especialista mais apropriada.
4.  Executa a cadeia escolhida e retorna a resposta final.

Este componente é o que torna nosso sistema inteligente e flexível.

## 0. Configuração Inicial e Recriação das Cadeias

Como nos notebooks anteriores, começamos inicializando nossos componentes principais (LLM, Embeddings, Pinecone).

Para manter este notebook autocontido e organizado, vamos redefinir nossas cadeias especializadas dentro de funções. Isso torna o código mais limpo e fácil de gerenciar.

In [ ]:
import os
from dotenv import load_dotenv

# Componentes do LangChain
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings

# Componentes do Router
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

# Carrega as variáveis do arquivo .env
load_dotenv()

# Pega as variáveis do ambiente
groq_api_key = os.getenv("GROQ_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
index_name = os.getenv("PINECONE_INDEX_NAME")

print("--- Inicializando Componentes ---")

# 1. LLM da Groq
llm = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0, api_key=groq_api_key)

# 2. Modelo de Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 3. Conexão com a base de dados vetorial (Pinecone)
vectorstore = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

print("LLM, Embeddings e VectorStore inicializados com sucesso!")

### Funções para Criar as Cadeias Especializadas

Aqui, encapsulamos a lógica de cada cadeia. Note que as cadeias de RAG (`info_local` e `logistica`) não são `LLMChain` diretas, pois elas precisam primeiro buscar dados no `vectorstore`. Para simplificar a integração com o `MultiPromptChain`, vamos criar uma estrutura que realiza a busca e depois chama a `LLMChain`.

In [ ]:
# Templates para cada cadeia
LOCAL_INFO_TEMPLATE = """Você é um assistente de turismo. Responda à pergunta do usuário de forma clara, usando APENAS as informações do contexto. Se a resposta não estiver no contexto, diga 'Não encontrei essa informação.'.
Contexto: {context}
Pergunta: {input}
Resposta:"""

LOGISTICS_TEMPLATE = """Você é um especialista em logística de viagens. Responda à pergunta do usuário sobre transporte de forma direta, usando as informações do contexto.
Contexto: {context}
Pergunta: {input}
Instruções de Transporte:"""

ITINERARY_TEMPLATE = """Você é um agente de viagens experiente. Crie um roteiro de viagem personalizado com base na solicitação do usuário. Use o contexto para sugerir pontos turísticos.
Contexto: {context}
Solicitação: {input}
Roteiro Detalhado:"""

TRANSLATION_TEMPLATE = """Você é um guia de idiomas para viajantes. Responda à solicitação do usuário fornecendo frases úteis e suas traduções.
Solicitação do Usuário: {input}
Frases e Traduções:"""

# Dicionário com os templates
prompt_infos = {
    "info_local": {"template": LOCAL_INFO_TEMPLATE, "description": "Ideal para responder perguntas sobre um local específico, como horário de funcionamento, localização ou detalhes de um ponto turístico."},
    "logistica": {"template": LOGISTICS_TEMPLATE, "description": "Use para responder perguntas sobre transporte, como chegar a um lugar, aeroportos e locomoção na cidade."},
    "roteiro_viagem": {"template": ITINERARY_TEMPLATE, "description": "Perfeito para quando o usuário pede um roteiro ou itinerário de viagem para uma cidade por um ou mais dias."},
    "traducao": {"template": TRANSLATION_TEMPLATE, "description": "Útil para fornecer traduções ou frases comuns em um idioma para viajantes."}
}

## 1. Construção do Roteador

O `LLMRouterChain` é o cérebro que decide qual caminho seguir. Ele usa um prompt especial que contém as descrições de cada cadeia especialista. A qualidade dessas descrições é o fator **mais importante** para o sucesso do roteador.

Depois, a `MultiPromptChain` junta tudo, atuando como o orquestrador geral.

In [ ]:
# 1. Criar as cadeias de destino (destination chains)
destination_chains = {}

for name, info in prompt_infos.items():
    # Define as variáveis de input corretas para cada template
    if name == "traducao":
        input_vars = ['input']
    else:
        input_vars = ['input', 'context']
        
    prompt = PromptTemplate(template=info['template'], input_variables=input_vars)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

print(f"{len(destination_chains)} cadeias de destino criadas: {list(destination_chains.keys())}")

# 2. Criar as informações para o roteador (nome, descrição)
destinations = [f"{name}: {info['description']}" for name, info in prompt_infos.items()]
destinations_str = "\n".join(destinations)

# 3. Criar o prompt do roteador
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

# 4. Criar a cadeia do roteador (o cérebro)
# Note que não criaremos mais a MultiPromptChain, apenas o roteador que decide.
router_chain = LLMRouterChain.from_llm(llm, router_prompt, verbose=True)

print("\nRouterChain configurada com sucesso!")
print("A MultiPromptChain não será mais usada. A lógica de execução estará na função run_query.")

## 2. Criando a Função de Execução com RAG

Agora, criamos uma função que une a lógica de RAG com o roteador. Esta função irá:
1. Receber a pergunta do usuário.
2. Usar o `vectorstore` para buscar o contexto relevante (RAG).
3. Chamar a `MultiPromptChain` com a pergunta **e** o contexto.

Isso garante que as cadeias que precisam de contexto (`info_local`, `logistica`, `roteiro_viagem`) o recebam. A cadeia `traducao`, que não usa contexto, simplesmente o ignorará.

In [ ]:
def run_query(query: str):
    """
    Executa uma consulta completa:
    1. Busca o contexto no Pinecone.
    2. Usa o router_chain para decidir qual cadeia usar.
    3. Executa a cadeia escolhida com o input e o contexto corretos.
    """
    print("--- Nova Consulta ---")
    print(f"Pergunta: {query}")
    
    # Passo 1: Buscar contexto relevante no Pinecone
    print("\nBuscando contexto no Pinecone...")
    docs = vectorstore.similarity_search(query, k=4)
    context = "\n\n".join([doc.page_content for doc in docs])
    
    # Passo 2: Usar o roteador para obter o nome da cadeia de destino
    print("\nUsando o roteador para decidir a cadeia...")
    route = router_chain.invoke({"input": query})
    destination_name = route['destination']
    
    # É importante verificar se o destino existe
    if destination_name in destination_chains:
        print(f"Rota escolhida: '{destination_name}'")
        
        # Passo 3: Selecionar e executar a cadeia de destino
        destination_chain = destination_chains[destination_name]
        
        # Prepara os inputs para a cadeia final
        inputs = {"input": query}
        # Adiciona o contexto apenas se a cadeia não for a de tradução
        if destination_name != "traducao":
            inputs["context"] = context
            
        print("\nExecutando a cadeia de destino...")
        result = destination_chain.invoke(inputs)
        return result['text']
    else:
        print("Erro: O roteador não conseguiu encontrar um destino válido.")
        return "Desculpe, não consegui processar sua pergunta."

## 3. Testes Finais

Vamos testar o sistema completo com diferentes tipos de perguntas para ver se o roteador escolhe a cadeia correta em cada caso. Observe os logs (`verbose=True`) para ver a decisão do `LLMRouterChain`.

In [ ]:
# Teste 1: Informação Local
query_info_local = "Qual o horário de funcionamento do Museu do Louvre?"
response = run_query(query_info_local)
print("\nResposta Final:\n", response)

In [ ]:
# Teste 2: Logística
query_logistica = "Como faço para ir do aeroporto Charles de Gaulle para o centro de Paris?"
response = run_query(query_logistica)
print("\nResposta Final:\n", response)

In [ ]:
# Teste 3: Roteiro de Viagem
query_roteiro = "Me dê um roteiro de 3 dias para o Rio de Janeiro, com foco em praias e vida noturna."
response = run_query(query_roteiro)
print("\nResposta Final:\n", response)

In [ ]:
# Teste 4: Tradução (não usará o contexto)
query_traducao = "Como se diz 'Onde fica o banheiro?' em francês?"
response = run_query(query_traducao)
print("\nResposta Final:\n", response)

## Conclusão do Projeto

Parabéns! Você construiu com sucesso um sistema de perguntas e respostas para turistas, utilizando uma arquitetura modular e poderosa com LangChain.

- **`01_Ingestao_Dados.ipynb`**: Você criou e populou sua base de conhecimento vetorial no Pinecone.
- **`02_Construcao_Chains.ipynb`**: Você desenvolveu e testou cadeias especialistas para tarefas específicas.
- **`03_Router_Chain.ipynb`**: Você orquestrou tudo com um roteador inteligente, que direciona as perguntas do usuário para o especialista correto, enriquecendo as respostas com informações atualizadas via RAG.

Este projeto é uma excelente base que pode ser expandida com novas cadeias, mais fontes de dados e até uma interface de usuário.